# Surgery to OMOP

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
surgery_IDEA4RC = pd.read_csv("./IDEA4RC-data/surgeryIDEA4RC.csv")
surgery_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Surgery to Measurement

Measurement concept id?

What to do with the vocabulary and modifier here:
Site of surgery on metastasis_soft tissue
Site of surgery on metastasis_distant lymph nodes
Site of surgery on metastasis_lung
Site of surgery on metastasis_bone
Site of surgery on metastasis_liver
Site of surgery on metastasis_pleura
Site of surgery on metastasis_peritoneum
Site of surgery on metastasis_brain
Site of surgery on metastasis_other viscera
Site of surgery on metastasis_unknown

In [1]:
df_tables=surgery_IDEA4RC

cols= {
    "Surgery intention": 0,
    "Type of surgical approach on Tumour": 0,
    "Margins after surgery": 0,
    "Tumor rupture": 36768904, #Since we have a vocabulary for a boolean, I will use it as a modifier
    "Surgical specimen Mitotic count": 0,
    "Surgical specimen grading only in untreated tumours": 0,
    "Reconstruction": 3010988,
    "Neck surgery": 0,
    "Laterality of the dissection": 0,
    "Surgery on M": 0,
    "Site of surgery on metastasis_soft tissue": 32945,
    "Site of surgery on metastasis_distant lymph nodes": 32946,
    "Site of surgery on metastasis_lung": 32947,
    "Site of surgery on metastasis_bone": 32948,
    "Site of surgery on metastasis_liver": 32949,
    "Site of surgery on metastasis_pleura": 32950,
    "Site of surgery on metastasis_peritoneum": 32951,
    "Site of surgery on metastasis_brain": 32952,
    "Site of surgery on metastasis_other viscera": 32952,
    "Site of surgery on metastasis_unknown": 32952,
    "Surgical complications (Clavien-Dindo Classification)": 4137797
}
sql =  """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_concept_id,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""

sql2 = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date,measurement_type_concept_id,value_as_number,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
"""


booleans=['Tumor rupture','Reconstruction','Neck surgery','Surgery on M',
             "Site of surgery on metastasis_soft tissue",
             "Site of surgery on metastasis_distant lymph nodes","Site of surgery on metastasis_lung",
             "Site of surgery on metastasis_bone","Site of surgery on metastasis_liver",
             "Site of surgery on metastasis_pleura","Site of surgery on metastasis_peritoneum",
             "Site of surgery on metastasis_brain","Site of surgery on metastasis_other viscera","Site of surgery on metastasis_unknown"]

boolValues=[4188539,4188540]
measurement_type_concept_id=38000280
for idx, row in df_tables.iterrows():
    for column in cols.keys():
        curIDEA.execute(query, (row['Episode Event Reference']))
        res=curIDEA.fetchone()
        person_id=res
        measurement_concept_id=cols.get(column)
        measurement_value=row[column]
        date=row['Date of surgery']
        cur.execute(sql, (person_id, measurement_concept_id, date, measurement_type_concept_id, measurement_value, measurement_event_id, NECESITAMOS_CODIGO))
        conn.commit()



SyntaxError: incomplete input (2863311216.py, line 56)

### Surgery to Procedure Ocurrence

Since this is a "treatment," we could create an Episode (which would involve storing data at a higher level), but since it is allowed in OMOP and we are instructed to do so in the mapping, we will simply use Procedure Occurrence (a lower level)

In [ ]:
queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s)
"""

sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_type_concept_id, provider_id)
    VALUES (%s, %s, %s,%s,%s, %s)
"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_concept_id=row['Surgery type']
    procedure_date=row['Date of surgery']
    procedure_type_concept_id=0 #Still dont know what to do with this one
    curIDEA.execute(queryDate, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    epId=cur.fetchone()
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    toTableEpisodeEventProcedureOcurrence(epId,newId)
    provider_id=row['surgeryHospital']
    cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_type_concept_id))
    conn.commit()
    if row['Surgery on M']==1 or row['Surgery on M']:
        newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
        toTableEpisodeEventProcedureOcurrence(epId,newId)
        procedure_date=row['Date of surgery on M']
        procedure_concept_id=
        cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_type_concept_id))
        conn.commit()

